#Install Packages

In [1]:
 pip install lxml

In [15]:
#import libraries
import requests 
from bs4 import BeautifulSoup
import csv
import pandas as pd
from itertools import zip_longest

In [16]:
#key words to check  if url is real url for privacy policy
key_words=['Privacy Policy','Terms and Conditions','Terms in Use','Terms & Conditions','Privacy Notice','privacy policy']

In [17]:
#list to append the content to it

texts=[]

#Enter URL

In [18]:
#Read the url

URL="https://www.edureka.co/privacy-policy"
result=requests.get(URL)


#Model Processing

In [19]:
#scrape the webpage and save its content

src=result.content

In [20]:
soup=BeautifulSoup(src,"lxml")

In [21]:
#Remove the pop up pages data

for i in range(50):
  try:
    pop=(soup.find('div',{'class':"modal-content"}))
    pop.decompose()
  except:
    True

In [22]:
#Remove the form page data

for i in range(50):
  try:
    form=soup.form
    form.decompose()
  except:
    True

In [23]:
#Remove the navigation bar data

for i in range(10):
  try:
    navigation=soup.nav
    navigation.decompose()
  except:
    True

In [24]:
#Remove the footer data

for i in range(10):
  try:
    footer=soup.footer
    footer.decompose()
  except:
    True

In [25]:
#Remove the header data

for i in range(10):
  try:
    header=soup.header
    header.decompose()
  except:
    True

In [26]:
#Remove the URL links that opens another pages data(if we dont remove it the componets of the other pages will be appear)

try:
    links=soup.a
    links.decompose()
except:
    True

In [27]:
#Remove the URL links that opens another pages data(if we dont remove it the componets of the other pages will be appear)

for i in range(1000):
    try:
        links=soup.a
        links.decompose()
    except:
        True

In [28]:
#find all paragraphs,lists, tables in the text

text=soup.find_all(["p","li","th","td"])

In [29]:
#append the data extracted to the list text to save the data without tags

for i in range(len(text)):
  texts.append(text[i].text)


In [30]:
#Transform data to dataframe (the dataframe will be printed at the end)

newdf=pd.DataFrame(texts)

In [31]:
#check if the url content contain at least one of key words or not

URL_index=[]
for i in range(len(newdf)):
  if any(word in newdf[0][i]for word in key_words):
      URL_index.append(i)


In [32]:
#check if the url is url for privacy policy or not

if any(word in URL for word in key_words):
  True
elif (len(URL_index) > 0):
  True
else :
  print("Invalid URL, not privacy policy page ")
  result=requests.get()


In [33]:
#make another  dataframe for our data(the dataframe to be processed)

df1=pd.DataFrame(texts)

In [34]:
#rename the column that contain the text to "text"

df1 = df1.rename(columns={0: 'text'})

In [35]:
#keep the index of paragraphs
df1['paragraph']=df1.index

In [36]:
#remove urls and clean html if it appeared in the paragraphs or list
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [37]:
# after removing the tags if there are tags in the text remove it
df1["text"] = df1.text.map(remove_URL)
df1["text"] = df1.text.map(cleanhtml)

tags_list = ['<p>' ,'</p>' , '<p*>',
             '<ul>','</ul>',
             '<li>','</li>',
             '<br>',
             '<strong>','</strong>',
             '<span*>','</span>',
             '<a href*>','</a>',
             '<em>','</em>','→']

for tag in tags_list:
  df1['text'].replace(to_replace=tag, value='', regex=True, inplace=True)

In [38]:
#prepare the function to remove the stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine
# has been programmed to ignore, both when indexing entries for searching and when retrieving them 
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [39]:
#apply the function
df1["text"] = df1.text.map(remove_stopwords)

In [40]:
#remove punctation
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [41]:
#lemmatization function
import nltk
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [42]:
#apply lemmatization
df1['text'] = df1.text.map(lemmatize_text)

In [43]:
# rejoin the words to scentences again
for i in range(len(df1)):
 df1.text[i] = ' '.join(df1.text[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
#remove punctations
df1["text"] = df1.text.map(remove_punct)


In [45]:
#remove the pargraphs that contain one word or zero words
small_index=[]
for i in range(len(df1)):
  if (len((df1['text'][i]).split())<2):
    small_index.append(i)

In [46]:

df1=df1.drop(df1.index[small_index])

In [47]:
#reset the index after droping step
df1.reset_index(drop=True, inplace=True)

In [48]:
#save the paragraph,list ,tables to another dataframe
df2=df1['text']
df2=pd.DataFrame(df2)

In [49]:
#import important libraries
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [50]:
#read category file (load file(11.csv))
df=pd.read_csv('11.csv')

In [51]:
#read group file (load file (11_G.csv))
dfgroup=pd.read_csv("11_G.csv")

In [52]:
#save the groups as string in new column grp
dfgroup['grp']=dfgroup['group']

In [53]:
#save the groups names as int in column group
dfgroup.group=dfgroup.group.replace({'A':0,'B':1,'C':2})

In [54]:
# save the group values as the labels for classification process
ygroup=dfgroup['group']

In [55]:
#list the group names in an array
group_names=np.array(['A','B','C'])

In [56]:
#if you find any word out of vocabulary give it the value OOV
oov_tok="<OOV>"

In [57]:
#save the category as string in new column cat

df['cat']=df['category']

In [58]:
#save the category names as int in column category

df.category=df.category.replace({"First Party Collection/Use":0,
"Third Party Sharing/Collection":1,"User Choice/Control":2,"User Access, Edit and Deletion":3,
"Data Retention":4,"Data Security":5,"Policy Change":6,"Do Not Track":7 ,"International and Specific Audiences":8,
"Other":9})

In [59]:
#list the category names in an array

target_names=['First Party Collection/Use','Third Party Sharing/Collection','User Choice/Control','User Access, Edit and Deletion','Data Retention','Data Security','Policy Change','Do Not Track','International and Specific Audiences','Other']

In [60]:
#list the group names in an array

category_name=np.array(target_names)


In [61]:
#removing url for input of the model
df["text"] = df.text.map(remove_URL)


In [62]:
#removing tags for input of the model
for tag in tags_list:
  df['text'].replace(to_replace=tag, value='', regex=True, inplace=True)

In [63]:
#removing stopwords for the model input
df["text"] = df.text.map(remove_stopwords)

In [64]:
#removing punctation for the model input
df["text"] = df.text.map(remove_punct)


In [65]:
#lemmatizing the input
df['text'] = df.text.map(lemmatize_text)

In [66]:
#rejoin the words to sentences
for i in range(len(df)):
  df.text[i] = ' '.join(df.text[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
#split the data to train and test data 
from sklearn.model_selection import train_test_split
x=df.text
y=df.category
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=4,shuffle=True)

In [68]:
#transform the input text into count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
x_train_count=count_vect.fit_transform(x_train)

In [69]:
#apply tfid transform on the text
from sklearn.feature_extraction.text import TfidfTransformer
tfid_transformer=TfidfTransformer()
x_train_tfidf=tfid_transformer.fit_transform(x_train_count)

In [70]:
#apply linear kernel svm model with 5 cv folds
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

text_clf=Pipeline([('tfidf',TfidfVectorizer()),
                   ('clf',SVC(kernel='linear',C=1,gamma='auto',probability=False,random_state=0)),])
text_clf.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 SVC(C=1, gamma='auto', kernel='linear', random_state=0))])

In [71]:
#make predictions on the test data to check accuracy
SVMpredictions=text_clf.predict(x_test)

In [72]:
#Accuracy report for category
from sklearn import metrics
print (metrics.classification_report(y_test,SVMpredictions,target_names=target_names))

                                      precision    recall  f1-score   support

          First Party Collection/Use       0.78      0.83      0.80       290
      Third Party Sharing/Collection       0.81      0.77      0.79       209
                 User Choice/Control       0.73      0.57      0.64        77
      User Access, Edit and Deletion       0.68      0.70      0.69        27
                      Data Retention       1.00      0.29      0.44         7
                       Data Security       0.89      0.82      0.85        39
                       Policy Change       0.94      0.91      0.92        32
                        Do Not Track       1.00      1.00      1.00         4
International and Specific Audiences       0.86      0.89      0.87        62
                               Other       0.74      0.79      0.76       201

                            accuracy                           0.79       948
                           macro avg       0.84      0.76     

In [73]:
#predict the category names for the scraped data
for i in range(len(df2)):
  pred=text_clf.predict(df2.iloc[i])
  (category_name[pred][0])

In [74]:
#split the data to train and test for the groups
from sklearn.model_selection import train_test_split
x=df.text
x_train2,x_test2,y_train2,y_test2=train_test_split(x,ygroup,test_size=0.25,random_state=4,shuffle=True)

In [75]:
#appply count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect2=CountVectorizer()
x_train_count2=count_vect.fit_transform(x_train2)

In [76]:
#apply tfid transformer 
from sklearn.feature_extraction.text import TfidfTransformer
tfid_transformer2=TfidfTransformer()
x_train_tfidf2=tfid_transformer2.fit_transform(x_train_count2)

In [77]:
#apply linear kernel svm with 5 fold cv to predict the groups
text_clf2=Pipeline([('tfidf2',TfidfVectorizer()),
                   ('clf2',SVC(kernel='linear',C=1,gamma='auto',probability=False,random_state=0)),])
text_clf2.fit(x_train2,y_train2)

Pipeline(steps=[('tfidf2', TfidfVectorizer()),
                ('clf2',
                 SVC(C=1, gamma='auto', kernel='linear', random_state=0))])

In [78]:
#predict the test data to check accuracy of groups
SVM2predictions=text_clf2.predict(x_test2)

In [79]:
#accuracy reports for the group
print (metrics.classification_report(y_test2,SVM2predictions,target_names=group_names))

              precision    recall  f1-score   support

           A       0.38      0.49      0.43       372
           B       0.32      0.31      0.31       311
           C       0.34      0.21      0.26       265

    accuracy                           0.35       948
   macro avg       0.34      0.34      0.33       948
weighted avg       0.35      0.35      0.34       948



In [80]:
##predict the group for scraped data
for i in range(len(df2)):
  pred2=text_clf2.predict(df2.iloc[i])
  (group_names[pred2][0])

In [82]:
#drop the text if the paragraph that has 0 or 1  words 
newdf=newdf.drop(newdf.index[small_index])

In [83]:
#reset the index again
newdf.reset_index(drop=True, inplace=True)

In [84]:
#import nltk to tokenize the predicted data to be shown
import nltk.data
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Prediction

In [85]:
print("--------------------------------------------------------------")
for i in range(len(df2)):
  pred=text_clf.predict(df2.iloc[i])
  tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
  data = newdf[0][i]
  print("Paragraph Number: ", i+1)
  print('\n'.join(tokenizer.tokenize(data)))
  print("Category :")
  print(category_name[pred][0])
  print("Group :")
  print(group_names[pred2][0])
  print("-----------------------------------------------------------")

--------------------------------------------------------------
Paragraph Number:  1

            This web site is owned and operated by Brain4ce Education Solutions Private Limited ("we", "our", or "us" or the "Company").
We understand and value your privacy.
We want to make your experience online satisfying and safe.
This privacy policy (the "Policy") governs information you provide to us or we learn from your use of this web site (the "Site") and constitutes a legal agreement between you, as the user of the Site, and the Company, as the owner of the Site.
The Policy will also tell you how we may collect, use, and in some instances share this information.
Our policies do not apply to third-party websites that are connected via links to our Site and may differ from other service offerings and you should carefully review the terms of service and this privacy notice before using these services.
Category :
Other
Group :
A
-----------------------------------------------------------
Paragra